In [6]:
from keras.layers import Input,Conv2D,Flatten,Dense,Reshape,Lambda,Conv3D,Ac
from keras.models import Model,Sequential
from keras import backend as K
from keras.optimizers import Adam,RMSprop

In [5]:
import numpy as np
import os
import pickle
from music21 import midi
from music21 import note,stream,duration,tempo

In [ ]:
class  MusicGenGan():
    def __init__(self,
                 input_dim
                 latent_dim,
                 critic_learning_rate,
                 generator_learning_rate,
                 optimizer,
                 batch_size,
                 n_tracks,
                 n_bars,
                 n_steps_per_bar,
                 n_pitches    
    ):
        self.input_dim=input_dim
        self.latent_dim=latent_dim
        self.critic_learning_rate=critic_learning_rate
        self.generator_learning_rate=generator_learning_rate
        self.optimizer=optimizer
        self.batch_size=batch_size
        self.n_tracks=n_tracks
        self.n_bars=n_bars
        self.n_steps_per_bar=n_steps_per_bar
        self.n_pitches=n_pitches
        
        
        self.d_losses=[]
        self.g_losses=[]
        
        self._build_critic()
        self._build_generator()
        self._build_adversarial()
        
        
    def conv(self,x,f,k,s,a,p):
        x=Conv3D(filters=f,
                kernel_size=k,strides=s)(x)
        if a=='relu':
            x=Activation(a)(x)
        elif a=='lrelu'
            x=LeakyReLU(x)